In [2]:
from torch.utils.data import DataLoader
import torch
import numpy as np
import time
import math
from data_loader.HHARDataset import HHARDataset
import pandas as pd
from torchvision import transforms
from PIL import Image
import cv2
import os

In [3]:
file_path = "D:\Dataset\HHAR\hhar_minmax_scaling_all.csv" 
save_class_path = "D:\Dataset\hhar256"
OVERLAPPING_WIN_LEN = 256 // 2
users = ['a','b','c','d','e','f','g','h','i']
class2id = {'bike':'0', 'sit':'1', 'stand':'2', 'walk':'3', 'stairsup':'4', 'stairsdown':'5'}
transform = transforms.Compose([ transforms.ToTensor() ,transforms.Resize((128, 128))])

df = pd.read_csv(file_path)

In [3]:
for user in users:
    df_a = df[ df['User'] == user]
    # 文件夹
    if not os.path.exists(os.path.join(save_class_path, user)):
        os.mkdir(os.path.join(save_class_path, user))
    for idx in range(max(len(df_a) // OVERLAPPING_WIN_LEN-1 , 0)):
        user = df_a.iloc[idx * OVERLAPPING_WIN_LEN, 9]
        model = df_a.iloc[idx * OVERLAPPING_WIN_LEN, 10]
        device = df_a.iloc[idx * OVERLAPPING_WIN_LEN, 11]
        class_label = df_a.iloc[idx * OVERLAPPING_WIN_LEN, 12]
        feature = df_a.iloc[idx * OVERLAPPING_WIN_LEN:(idx + 2) * OVERLAPPING_WIN_LEN,3:9].values
        feature = feature.T
        # reshape
        f = transform(feature)
        f = f.expand(3,-1,-1)
        f = f.reshape(128,128,3)
        f = f.numpy()
        # print(f.shape)
        # 归一化转图像
        f =  ((f - f.min()) * (1 / (f.max() - f.min()) * 255)).astype(np.uint8)
        image = Image.fromarray(f)
        if not os.path.exists(os.path.join(save_class_path, user, str( class2id[class_label]))):
            os.mkdir(os.path.join(save_class_path, user, str( class2id[class_label])))
        save_file_path = os.path.join( save_class_path, user, str( class2id[class_label]) , model + str(idx)+'.jpg')
        image.save(save_file_path)
    print("user:%s done!"%(user))
  
    



user:a done!
user:b done!
user:c done!
user:d done!
user:e done!
user:f done!
user:g done!
user:h done!
user:i done!


In [6]:
num_list = []
for user in users:
    df_a = df[ df['User'] == user]
    user_data = []
    for idx in range(max(len(df_a) // OVERLAPPING_WIN_LEN-1 , 0)):
        user = df_a.iloc[idx * OVERLAPPING_WIN_LEN, 9]
        model = df_a.iloc[idx * OVERLAPPING_WIN_LEN, 10]
        device = df_a.iloc[idx * OVERLAPPING_WIN_LEN, 11]
        class_label = df_a.iloc[idx * OVERLAPPING_WIN_LEN, 12]
        feature = df_a.iloc[idx * OVERLAPPING_WIN_LEN:(idx + 2) * OVERLAPPING_WIN_LEN,3:9].values
        feature = feature.T
        user_data.append(feature)
    print(len(user_data))
    num_list.append(len(user_data))
print("mean", np.mean(num_list))
  

11083
11773
10917
10967
12073
10901
8781
8699
8981
mean 10463.888888888889


使用Metasense的HHAR数据loader加载数据

In [21]:
file_path = "D:\Dataset\HHAR\hhar_minmax_scaling_all.csv" 
'nexus5x.neuxs5x_1.a'
mydata = HHARDataset(file=file_path, model='nexus5x', device='neuxs5x_1', user='a',
                                              complementary=True )
print(mydata.__getitem__(0)[0].shape )

datssets = mydata.get_datasets_per_domain()
for idx ,  d in enumerate( datssets):
    print(len(d), mydata.domain_superset[idx])

Valid domains:[('nexus4', 'b'), ('nexus4', 'c'), ('nexus4', 'd'), ('nexus4', 'e'), ('nexus4', 'f'), ('s3', 'b'), ('s3', 'c'), ('s3', 'd'), ('s3', 'e'), ('s3', 'f'), ('s3mini', 'b'), ('s3mini', 'c'), ('s3mini', 'd'), ('s3mini', 'e'), ('s3mini', 'f'), ('lgwatch', 'b'), ('lgwatch', 'c'), ('lgwatch', 'd'), ('lgwatch', 'e'), ('lgwatch', 'f')]
Loading data done with rows:10636800	Preprocessing:530.893903	Total Time:2455.451859
